In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.special

In [12]:
pip install permpy

  Found existing installation: pytest 5.2.1
    Uninstalling pytest-5.2.1:
      Successfully uninstalled pytest-5.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
def plot_permutation(P):
    n = len(P)
    x_values = [i for i in range(n)]
    y_values = P
    plt.scatter(x_values,y_values, label = str(P))

In [3]:
def unrank_permutation(n_list,d):
    n = len(n_list)
    nn_list = [h for h in n_list]
    if d == 0:
        return nn_list
    for k in range(n):
        if d < (k+1)*np.math.factorial(n-1):
            nn_list.remove(n_list[k])
            return [n_list[k]] + unrank_permutation(nn_list,d - k*np.math.factorial(n-1)) 

In [4]:
def unrank_binomial(n,k,d):
    if n == 0:
        return ""
    C = scipy.special.binom(n-1,k)
    if d < C:
        return "0" + unrank_binomial(n-1,k,d)
    else:
        return "1" + unrank_binomial(n-1,k-1,d-C)

In [5]:
def relative_order(p):
    '''takes a list of n distinct integers and returns the permutation from 0 to n-1 with the same relative order.'''
    n = len(p)
    SP = sorted(p)
    output = [0]*n
    for i in range(n):
        ind = SP.index(p[i])
        output[i] = ind
    return output

In [6]:
def avoids_231(permutation):
    """determines (True or False) if a permutation avoids the pattern 231"""
    n = len(permutation)
    if n<2:
        return True
    max_ind = 0
    for i in range(1,n):
        if permutation[max_ind] < permutation[i]:
            max_ind = i
    if avoids_231(permutation[:max_ind]):
        if avoids_231(permutation[max_ind+1:]):
            minL = 10000
            for j in range(max_ind):
                if permutation[j] < minL:
                    minL = permutation[j]
            for j in range(max_ind+1,n):
                if permutation[j] > minL:
                    return False
            return True
    return False
    

In [13]:
def avoids_321_slow(permutation):
    """determines (True or False) if a permutation avoids the pattern 321"""
    n = len(permutation)
    for i in range(1,n-2):
        for j in range(i,n-1):
            if permutation[i] > permutation[j]:
                for k in range(j,n):
                    if permutation[j]>permutation[k]:
                        return False
    return True

In [15]:
def ALL_321_Avoiders(n):
    output = []
    fn = np.math.factorial(n)
    for i in range(fn):
        L = [k for k in range(n)]
        p = unrank_permutation[k,i]
        if avoids_321_slow(p):
            output = output + [p]
    return output

In [7]:
def count_different_patterns(P,k):
    """counts the number of different patterns of length k in permutation P"""
    count = 0
    pattern_hash = defaultdict(int)
    n = len(P)
    C = scipy.special.binom(n,k)
    for i in range(int(C)):
        fbs = unrank_binomial(n,k,i)
        rel_order = []
        for j in range(n):
            if fbs[j] == "1":
                rel_order = rel_order + [P[j]]
        RO = str(relative_order(rel_order))
        if pattern_hash[RO] == 0:
            count = count + 1
        pattern_hash[RO] = pattern_hash[RO] + 1
    return count

In [8]:
def count_different_231_avoiders(P,k):
    """counts the number of different 231-avoiding patterns of length k in permutation P"""
    count = 0
    pattern_hash = defaultdict(int)
    n = len(P)
    C = scipy.special.binom(n,k)
    for i in range(int(C)):
        fbs = unrank_binomial(n,k,i)
        rel_order = []
        for j in range(n):
            if fbs[j] == "1":
                rel_order = rel_order + [P[j]]
        RO = str(relative_order(rel_order))
        if pattern_hash[RO] == 0:
            if avoids_231(rel_order):
                count = count + 1
                pattern_hash[RO] = pattern_hash[RO] + 1
    return count

In [9]:
count_different_231_avoiders([7,2,1,6,8,9,3,0,5,4],6)

10

In [10]:
count_different_patterns([7,2,1,6,8,9,3,0,5,4],6)

59

In [29]:
def crossover1(P1,P2):
    """random entry fill in the gaps"""
    n = len(P1)
    offspring = [-1]*n
    r = random.random()
    if r < 0.5:
        offspring[0] = P1[0]
    else:
        offspring[0] = P2[0]
    for i in range(1,n):
        r = random.random()
        if r < 0.5:
            j = i
            while P1[j] in offspring:
                j = (j+1) % n
            offspring[i] = P1[j]
        else:
            j = i
            while P2[j] in offspring:
                j = (j+1) % n
            offspring[i] = P2[j]
    return offspring

In [30]:
def crossover2(P1,P2):
    """cut and crossfill"""
    n = len(P1)
    child1 = [-1]*n
    child2 = [-1]*n
    r = random.randint(1,n-1)
    for i in range(r):
        child1[i] = P1[i]
        child2[i] = P2[i]
    j = r
    while j < n:
        for k in range(n):
            if P2[k] not in child1:
                child1[j] = P2[k]
                j=j+1
    j = r
    while j < n:
        for k in range(n):
            if P1[k] not in child2:
                child2[j] = P1[k]
                j = j+1
    return [child1,child2]

In [31]:
def crossover3(P1,P2):
    """cut and cross-pattern"""
    n = len(P1)
    child1 = [-1]*n
    child2 = [-1]*n
    r = random.randint(1,n-1)
    for i in range(r):
        child1[i] = P1[i]
        child2[i] = P2[i]
    RO1 = relative_order(P1[r:])
    RO2 = relative_order(P2[r:])
    k = 0
    for i in range(n):
        if i not in child1:
            child1[r+RO2.index(k)] = i
            k = k+1
    k=0
    for i in range(n):
        if i not in child2:
            child2[r+RO1.index(k)] = i
            k = k+1
    return [child1,child2]

In [32]:
def mut1(Perm,r1,r2):
    """helper function for mutation1"""
    n = len(Perm)
    output = [i for i in Perm]
    if r1 == r2:
        return(Perm)
    output[r1+1]=Perm[r2]
    for i in range(r1+2,r2+1):
        output[i] = Perm[i-1]
    return output


def mutation1(Perm):
    """does a single random transposition to a permutation"""
    n = len(Perm)
    r1 = random.randint(0,n-1)
    r2 = random.randint(0,n-1)
    if r1<=r2:
        return mut1(Perm,r1,r2)
    else:
        REV = [Perm[n-1-j] for j in range(n)]
        output = mut1(REV,r2,r1)
        return [output[n-1-j] for j in range(n)]

In [35]:
def Evolutionary_Algorithm_random_universality_231(n,k,p,g,m,pp):
    """attempts to select the best n-length permutation based on how many different k-patterns are found"""
    """p is size of population, g is number of fitness evaluations until algorithm terminates, m is mutation rate, pp is the size of the 'parent pool' to select parents"""
    #initialize population and compute fitness for each individual
    POP = []
    for j in range(p):
        r = random.randint(0,np.math.factorial(n)-1)
        P = unrank_permutation([i for i in range(n)],r)
        fitness = count_different_231_avoiders(P,k)
        POP = POP + [[fitness,P]]
    #sort initial population from most fit to least fit.
    POP.sort()
    POP.reverse()
    #initialize a counter for the number of fitness evaluations
    fe = 0
    #start the loop
    while(fe<g+1):
        #randomly select pp individuals from the POPulation to form the parent pool
        randomset = random.sample(POP,pp)
        #select the top 2 from the parent pool to be parents and crossover
        randomset.sort()
        P1 = randomset[-1]
        P2 = randomset[-2]
        P3 = randomset[-3]
        P4 = randomset[-4]
        crossover_type = random.randint(1,3)
        if crossover_type == 1:
            child1 = crossover1(P1[1],P4[1])
            child2 = crossover1(P2[1],P3[1])
        if crossover_type == 2:
            [child1,child2] = crossover2(P1[1],P2[1])
        if crossover_type == 3:
            [child1,child2] = crossover3(P1[1],P2[1])
        fitness1 = count_different_231_avoiders(child1,k)
        fitness2 = count_different_231_avoiders(child2,k)
        fe = fe+2
        #apply mutation to the offspring at a m mutation rate
        mr = random.random()
        if mr < m: 
            notimproved = True
            while notimproved:
                mchild1 = mutation1(child1)
                mchild2 = mutation1(child2)
                mfitness1 = count_different_231_avoiders(child1,k)
                mfitness2 = count_different_231_avoiders(child2,k)
                fe = fe+2
                if mfitness1 >= fitness1:
                    notimproved = False
                    child1 = mchild1
                    fitness1 = mfitness1
                if mfitness2 >= fitness2:
                    notimproved = False
                    child2 = mchild2
                    fitness2 = mfitness2            
                mr = random.random()
        #compute fitness for each child and increment the fitness evaluation counter
        #include the offspring into the population, resort and delete the 2 weakest individuals.
        POP = POP + [[fitness1,child1]]
        POP = POP + [[fitness2,child2]]
        POP.sort()
        POP.reverse()
        POP = POP[:p]
#        if fe % int(g/10) == 0:
#            print([fe],[POP[0]],[POP[j][0] for j in range(10)],[POP[-1][0]])
#            plot_permutation(POP[0][1])
#            plt.show()
    return POP

# Experiments:

* Input: Evolutionary_Algorithm2_universality(13,7,1000,10000,0.8,25): This took around 10 minutes and maxed out at 1007. The correct answer according to Michael's chart is 1094. The maximal permutation is: [9, 2, 6, 12, 0, 8, 4, 10, 1, 5, 11, 7, 3]. The population was 1000, and all 1000 individuals were identical. Let's try with a larger population and a larger parent pool and a larger number of fitness evaluations.

* Input: Evolutionary_Algorithm2_universality(13,7,5000,20000,0.8,30): This took around 20-30 minutes? and maxed out at 1074. The correct answer according to Michael's chart is 1094. The maximal permutation is: [4, 9, 1, 6, 11, 3, 7, 0, 10, 5, 12, 2, 8].

* Input: Evolutionary_Algorithm1_universality(13,7,1000,10000,0.8,25): This took around 10 minutes and maxed out at 964. The correct answer according to Michael's chart is 1094. The maximal permutation is: [7, 1, 10, 3, 6, 11, 0, 4, 8, 12, 2, 9, 5]. Using crossover 1 makes for a slower evolution process rather than crossover 2 but maybe that is okay.

* Input: Evolutionary_Algorithm1_universality(13,7,5000,20000,0.8,30): This took around 20-30 minutes? and maxed out at 979. The correct answer according to Michael's chart is 1094. The maximal permutation is: [9, 1, 5, 11, 3, 7, 0, 4, 8, 12, 2, 10, 6].


* 3: 1063
* 3: 1066

* Evolutionary_Algorithm_random_universality(13,7,5000,20000,0.8,30): 1066
* EA = Evolutionary_Algorithm_random_universality(13,7,7000,50000,0.8,40): 1065


In [36]:
U = [[0]*20 for i in range(20)]

In [ ]:
for m in range(1,14):
    for k in range(1,14):
        n = k+m
        EA = Evolutionary_Algorithm_random_universality_231(n,m,1000,10000,0.8,25)
        U[m][k] = EA[0][0]
        print(U[m][k],(m,k))

1 (1, 1)
1 (1, 2)
1 (1, 3)
1 (1, 4)
1 (1, 5)
1 (1, 6)
1 (1, 7)
1 (1, 8)
1 (1, 9)
1 (1, 10)
1 (1, 11)
1 (1, 12)
1 (1, 13)
2 (2, 1)
2 (2, 2)
2 (2, 3)
2 (2, 4)
2 (2, 5)
2 (2, 6)
2 (2, 7)
2 (2, 8)
2 (2, 9)
2 (2, 10)
2 (2, 11)
2 (2, 12)
2 (2, 13)
3 (3, 1)
5 (3, 2)
5 (3, 3)
5 (3, 4)
5 (3, 5)
5 (3, 6)
5 (3, 7)
5 (3, 8)
5 (3, 9)
5 (3, 10)
5 (3, 11)
5 (3, 12)
5 (3, 13)
4 (4, 1)
8 (4, 2)
12 (4, 3)
14 (4, 4)
14 (4, 5)
14 (4, 6)
14 (4, 7)
14 (4, 8)
14 (4, 9)
14 (4, 10)
14 (4, 11)
14 (4, 12)
14 (4, 13)
5 (5, 1)
13 (5, 2)
22 (5, 3)
31 (5, 4)
37 (5, 5)
42 (5, 6)
42 (5, 7)
42 (5, 8)
42 (5, 9)
42 (5, 10)
42 (5, 11)
42 (5, 12)
42 (5, 13)
6 (6, 1)
18 (6, 2)
38 (6, 3)
60 (6, 4)
83 (6, 5)
99 (6, 6)
118 (6, 7)
126 (6, 8)
130 (6, 9)
132 (6, 10)
